# Sprint 12 Tasca 2. Tasca de web scraping

## Nivell 1

## Exercici 1.  Realitza web scraping d'una pàgina de la borsa de Madrid (https://www.bolsamadrid.es) utilitzant BeautifulSoup i Selenium.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
from bs4 import BeautifulSoup
import requests

def get_soup(url):
    res = requests.get(url)
    return BeautifulSoup(res.content, "html.parser")

pag = 'https://www.bolsamadrid.es/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES0157097017&ClvEmis=57097'


### Soup conté l'HTML de la pàgina web, ara només hem d'extreure la data que ens interesa

In [3]:
soup = get_soup(pag)

In [4]:
## Amb 'get_text()' podem extreure tot el text de la pàgina sense etiquetes HTML però si només volem extreure alguna info de la pàgina podem filtrat per etiqueta o per id, per exemple
tot_text = soup.get_text()

In [5]:
## Amb '.title.get_text()' podem extreure el títol de la pàgina
titol = soup.title.get_text()
print('Títol: ', titol)

Títol:  
	Bolsa de Madrid - Ficha de ALMIRALL, S.A.



In [6]:
## Amb 'soup.p' podem extreure la descripció de l'empresa
descripcio = soup.p
descripcio
print('Descripció: ', descripcio)

Descripció:  <p> Almirall es una compañía biofarmacéutica global enfocada en la salud de la piel. Colaboramos con científicos y profesionales de la salud para abordar las necesidades del paciente a través de la ciencia con el fin de mejorar sus vidas. Nuestro propósito noble guía nuestro trabajo: “Transform the patients' world by helping them realize their hopes and dreams for a healthy life”. Invertimos en productos de dermatología médica diferenciados y pioneros para llevar nuestras soluciones innovadoras a los pacientes que lo necesitan. </p>


In [7]:
## Amb 'soup.find' podem extreure info per id
print()
capital = soup.find(id="ctl00_Contenido_CapAdmDat").get_text()
print('Capital admitido: ', capital)


Capital admitido:  21.573.216,24 Euros 


In [8]:
## Utilitzen find + regular expressions per extreure els preus
preus = soup.find(id="ctl00_Contenido_tblPrecios")
print('Últims preus: ', preus)

Últims preus:  <table cellpadding="3" cellspacing="0" class="TblPort" id="ctl00_Contenido_tblPrecios" width="100%">
<tr align="center">
<th scope="col">Fecha</th><th scope="col">Hora</th><th scope="col">Cierre</th><th scope="col">Ref.</th><th scope="col">% Dif.</th><th scope="col">Últ.</th><th scope="col">Máx.</th><th scope="col">Mín.</th><th scope="col">Medio</th><th scope="col">Volumen</th><th class="Ult" scope="col">Efectivo</th>
</tr><tr align="right">
<td align="center" class="enlacePosiciones" data-categoryga="Posiciones Acciones" data-labelga="ALMIRALL">11/02/2022</td><td align="center">Cierre</td><td>10,6200</td><td>10,8400</td><td class="DifClBj">-2,03</td><td>10,6200</td><td>10,8000</td><td>10,6200</td><td>10,7120</td><td>174.262</td><td class="Ult">1.866.688,36</td>
</tr>
</table>


In [9]:
import re
# regex to extract required strings
tag = 'td'
reg_str = "<" + tag + ">(.*?)</" + tag + ">"
preuss = re.findall(reg_str, str(preus))

In [10]:
preuss

['10,6200', '10,8400', '10,6200', '10,8000', '10,6200', '10,7120', '174.262']

In [11]:
preu_cierre = preuss[0]
preu_max = preuss[3]
preu_min = preuss[4]
preu_medio = preuss[5]
volumen = preuss[6]
print('Preu tancament: ', preu_cierre)
print('Preu màxim: ', preu_max)
print('Preu mínim: ', preu_min)
print('Preu mitjà: ', preu_medio)
print('Volumen: ', volumen)

Preu tancament:  10,6200
Preu màxim:  10,8000
Preu mínim:  10,6200
Preu mitjà:  10,7120
Volumen:  174.262


In [12]:
reg_str_2 = "<td align=\"center\" class=\"enlacePosiciones\" data-categoryga=\"Posiciones Acciones\" data-labelga=\"(.*?)\">(.*?)</td>"
data = re.findall(reg_str_2, str(preus))

In [13]:
nom = data[0][0]
fecha = data[0][1]
print('Nom empresa: ', nom)
print('Data: ', fecha)

Nom empresa:  ALMIRALL
Data:  11/02/2022


### Guardem la info en arrays per poder crear després un df amb dades d'altres empreses

In [14]:
arr_empresa = [nom]
arr_descripcio =[descripcio]
arr_fecha = [fecha]
arr_capital = [capital]
arr_cierre = [preu_cierre]
arr_max = [preu_max]
arr_min = [preu_min]
arr_medio = [preu_medio]
arr_vol = [volumen]

### SELENIUM

In [15]:
import  os
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#from selenium.webdriver.common.keys import Keys
print ('################ Test start ################')
loginUrl  = pag 
loginName = '1001'
loginPass = '1001'
 
options = webdriver.ChromeOptions()
options.add_argument ('- sin cabeza') # Habilita el modo sin cabeza
options.add_argument ('- disable-gpu') # Desactiva la aceleración de la GPU de la tarjeta gráfica, la recomendación oficial es desactivar: https: # bugs.chromium.org / p / chromium / issues / detail? id = 737678
 
 # En el modo sin cabeza, es mejor establecer el tamaño específico, ya que el valor predeterminado se maximiza, después de la captura de pantalla final, es muy pequeño, equivalente a incompleto
 # options.add_argument ('- start-maximized') # Maximizar al inicio
 
options.add_argument ('- no-sandbox') # Desactivar sandbox
options.add_argument ('- disable-plugins') # Desactivar complementos
options.add_argument ('- disable-popup-block') # deshabilitar el aviso de bloqueo
options.add_argument ('- ignore-certificate-errors') # Ignorar aviso de advertencia de error de certificado
 # options.add_argument ('- test-type', '- ignore-certificate-errors') # Ignora el aviso de advertencia de error del certificado, ¡sin efecto!
 # options.add_experimental_option ('excludeSwitches', ['ignore-certificate-errors']) # Ignora el aviso de advertencia de error del certificado, ¡sin efecto!
options.add_argument ('- allow-running-insecure-content') # Desactivar solicitud insegura
options.add_argument ('- disable-infobars') # Desactiva el mensaje "Chrome está siendo controlado por un software de prueba automático"
 
driver  = webdriver.Chrome(chrome_options=options)  # Firefox()  , Ie()
 # Establecer el tamaño y la ubicación del navegador
driver.set_window_position (50,50); # La coordenada de la ventana en relación con la esquina superior izquierda de la pantalla (0,0)
driver.set_window_size (1440,900); # tamaño de ventana 
 
print ('---> URL de visita: ' + loginUrl)
driver.get(loginUrl)

# Imprimir título de la página  actual
print('---> Título de la pág.: '  + driver.title)
 
# Buscar otra empresa en el buscador
print('---> Buscar otra empresa en el buscador')
elem = driver.find_element_by_name("busq")
elem.send_keys("Banco de Sabadell")
elem.send_keys(Keys.RETURN)
continue_link = driver.find_element_by_link_text('BANCO DE SABADELL, S.A.')
continue_link.send_keys(Keys.RETURN)

# Imprimir título de la segunda página
print('---> Título de la segunda pág.: ' + driver.title)

## Extreure info de la pàgina per afegir-la a les nostres arrays
soup = get_soup(driver.current_url)

arr_descripcio.append(soup.p)
arr_capital.append(soup.find(id="ctl00_Contenido_CapAdmDat").get_text())

preus = soup.find(id="ctl00_Contenido_tblPrecios")
tag = 'td'
reg_str = "<" + tag + ">(.*?)</" + tag + ">"
preuss = re.findall(reg_str, str(preus))

reg_str_2 = "<td align=\"center\" class=\"enlacePosiciones\" data-categoryga=\"Posiciones Acciones\" data-labelga=\"(.*?)\">(.*?)</td>"
data = re.findall(reg_str_2, str(preus))

arr_empresa.append(data[0][0])
arr_fecha.append(data[0][1])
arr_cierre.append(preuss[0])
arr_max.append(preuss[3])
arr_min.append(preuss[4])
arr_medio.append(preuss[5])
arr_vol.append(preuss[6])

# Ir al gráfico de 5 años
print('---> Ir al gráfico de 5 años')
graf_5 = driver.find_element_by_partial_link_text('5 años')
graf_5.send_keys(Keys.RETURN)

# Imprimir título de la tercera página
print('---> Título de la tercera pág.: ' + driver.title)

# Buscar otra empresa en el buscador
print('---> Buscar otra empresa en el buscador')
elem = driver.find_element_by_name("busq")
elem.send_keys("ACCIONA,S.A.")
elem.send_keys(Keys.RETURN)
continue_link = driver.find_element_by_link_text('ACCIONA,S.A.')
continue_link.send_keys(Keys.RETURN)

# Imprimir título de la cuarta página
print('---> Título de la cuarta pág.: ' + driver.title)

## Extreure info de la pàgina per afegir-la a les nostres arrays
soup = get_soup(driver.current_url)

arr_descripcio.append(soup.p)
arr_capital.append(soup.find(id="ctl00_Contenido_CapAdmDat").get_text())

preus = soup.find(id="ctl00_Contenido_tblPrecios")
tag = 'td'
reg_str = "<" + tag + ">(.*?)</" + tag + ">"
preuss = re.findall(reg_str, str(preus))

reg_str_2 = "<td align=\"center\" class=\"enlacePosiciones\" data-categoryga=\"Posiciones Acciones\" data-labelga=\"(.*?)\">(.*?)</td>"
data = re.findall(reg_str_2, str(preus))

arr_empresa.append(data[0][0])
arr_fecha.append(data[0][1])
arr_cierre.append(preuss[0])
arr_max.append(preuss[3])
arr_min.append(preuss[4])
arr_medio.append(preuss[5])
arr_vol.append(preuss[6])

print ('---> Cerrar el navegador')
driver.close()
print ('################ Fin de la prueba ################')



################ Test start ################
---> URL de visita: https://www.bolsamadrid.es/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES0157097017&ClvEmis=57097
---> Título de la pág.: Bolsa de Madrid - Ficha de ALMIRALL, S.A.
---> Buscar otra empresa en el buscador
---> Título de la segunda pág.: Bolsa de Madrid - Ficha de BANCO DE SABADELL, S.A.
---> Ir al gráfico de 5 años
---> Título de la tercera pág.: Bolsa de Madrid
---> Buscar otra empresa en el buscador
---> Título de la cuarta pág.: Bolsa de Madrid - Ficha de ACCIONA,S.A.
---> Cerrar el navegador
################ Fin de la prueba ################


### Creem el df amb les dades que hem recollit

In [23]:
df = pd.DataFrame(arr_empresa,columns =['Empresa'])
df['Descripcio'] = arr_descripcio
df['Data'] = arr_fecha
df['Capital'] = arr_capital
df['Preu_cierre'] = arr_cierre 
df['Preu_max'] = arr_max
df['Preu_min'] = arr_min
df['Preu_medio'] = arr_medio
df['Volumen'] = arr_vol 
df

,Empresa,Descripcio,Data,Capital,Preu_cierre,Preu_max,Preu_min,Preu_medio,Volumen
0,ALMIRALL,[ Almirall es una compañía biofarmacéutica glo...,11/02/2022,"21.573.216,24 Euros","10,6200","10,8000","10,6200","10,7120",174.262
1,BANCO DE SABADELL,[Banco Sabadell encabeza el cuarto grupo banca...,11/02/2022,"703.370.587,62 Euros","0,9432","0,9484","0,9222","0,9388",48.392.926
2,ACCIONA,[ACCIONA es una de las principales corporacion...,11/02/2022,"54.856.653,00 Euros","144,0000","145,6000","141,9000","144,0652",101.375


## Nivell 2

## Exercici 2. Documenta en un word el teu conjunt de dades generat amb la informació que tenen els diferents arxius de Kaggle.

**Atenció**    *En lloc de crear un word, he inclòs la informació en les següent cel·les formatejant-la amb l'ajuda de Markdown*

#### Context

Aquest conjunt de dades inclou informació de tres empreses que es troben el l'**IBEX 35**: Almirall, Acciona i Banco de Sabadell. Entre les dades que s'han extret hi ha una descripció de l'empresa, el seu capital i volum, i els valors màxim, mínim, mitjà i de tancament del preu en el día de la consulta.

#### Contingut

- Empresa: El nom oficial de l'empresa.
- Descripcio: Petita descripció de l'empresa.
- Data: El dia, mes i any (en format: dd/mm/yyyy) en que s'han consultat els valors (no fa referència al moment en que s'ha fet l'scrapping, sino en que s'han publicat les dades al web, ja que en dissabte i diumenge no hi ha actualitzacions al web, per tant, la data dels dissabtes i diumenges serà la del divendres anterior.
- Capital: El capital social de l'empresa (en €).
- Preu_cierre: El preu de tancament, és a dir, el preu de l'acció en el moment què s'ha tancat la borsa en el dia que s'indica a la Data.
- Preu_max: El preu màxim de l'acció assollit durant el dia que s'indica a la Data.
- Preu_min: El preu mínim de l'acció assollit durant el dia que s'indica a la Data.
- Preu_medio: El preu mirtjà al que ha estat l'acció durant el dia que s'indica a la Data.
- Volumen: El nombre d'accions que han estat objecte de compra venda durant el dia que s'indica a la Data.


#### Agraïment

Les dades han estat srapejades de la pàgina web de la Bolsa de Madrid: https://www.bolsamadrid.es/. 

## Nivell 3

## Exercici 3. Tria una página web que tu vulguis i realitza web scraping mitjançant la llibreria Scrapy. 



Després de seguir el tutorial de https://docs.scrapy.org/en/latest/intro/tutorial.html aquests son els documents JSON amb les dades que hem descarregat de la pàgina quotes.

In [30]:
import json

with open('quotes.json') as f:
    json_data = json.load(f)
    
with open('author.json') as f:
    json_data_2 = json.load(f)

In [31]:
json_data[:10]

[{'text': '“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
  'author': 'Albert Einstein'},
 {'text': '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
  'author': 'J.K. Rowling'},
 {'text': '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
  'author': 'Albert Einstein'},
 {'text': '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
  'author': 'Jane Austen'},
 {'text': "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
  'author': 'Marilyn Monroe'},
 {'text': '“Try not to become a man of success. Rather become a man of value.”',
  'author': 'Albert Einstein'},
 {'text': '“It is better to be hated for what you are than to be loved for what you are not.”',
  'author': 'André Gide

In [32]:
json_data_2[:10]

[{'name': 'Thomas A. Edison',
  'birthdate': 'February 11, 1847',
  'bio': 'Thomas Alva Edison was an American inventor, scientist and businessman who developed many devices that greatly influenced life around the world, including the phonograph, the motion picture camera, and a long-lasting, practical electric light bulb. Dubbed "The Wizard of Menlo Park" (now Edison, New Jersey) by a newspaper reporter, he was one of the first inventors to apply the principles of mass production and large teamwork to the process of invention, and therefore is often credited with the creation of the first industrial research laboratory.Edison is considered one of the most prolific inventors in history, holding 1,093 U.S. patents in his name, as well as many patents in the United Kingdom, France and Germany. He is credited with numerous inventions that contributed to mass communication and, in particular, telecommunications. His advanced work in these fields was an outgrowth of his early career as a te

### Ara podem crear un df unint les dades dels dos arxius

In [35]:
df_test_author = pd.DataFrame.from_dict(json_data)
df_test_author.head()

,text,author
0,“The world as we have created it is a process ...,Albert Einstein
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling
2,“There are only two ways to live your life. On...,Albert Einstein
3,"“The person, be it gentleman or lady, who has ...",Jane Austen
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe


In [36]:
df_bio_author = pd.DataFrame.from_dict(json_data_2)
df_bio_author.head()

,name,birthdate,bio
0,Thomas A. Edison,"February 11, 1847","Thomas Alva Edison was an American inventor, s..."
1,André Gide,"November 22, 1869",André Paul Guillaume Gide was a French author ...
2,Eleanor Roosevelt,"October 11, 1884",Anna Eleanor Roosevelt was an American politic...
3,Jane Austen,"December 16, 1775",Jane Austen was an English novelist whose work...
4,J.K. Rowling,"July 31, 1965",See also: Robert GalbraithAlthough she writes ...


In [41]:
df_bio_author_2 = df_bio_author.rename(columns={"name": "author"})
df_bio_author_2.head()

,author,birthdate,bio
0,Thomas A. Edison,"February 11, 1847","Thomas Alva Edison was an American inventor, s..."
1,André Gide,"November 22, 1869",André Paul Guillaume Gide was a French author ...
2,Eleanor Roosevelt,"October 11, 1884",Anna Eleanor Roosevelt was an American politic...
3,Jane Austen,"December 16, 1775",Jane Austen was an English novelist whose work...
4,J.K. Rowling,"July 31, 1965",See also: Robert GalbraithAlthough she writes ...


In [43]:
df_final = pd.merge(df_test_author, df_bio_author_2, on='author')

### Aquest és el resultat: un conjunt de dades amb 99 cites de 49 autors diferents, amb la seva data de naixement i una petita biografia.

L'autor amb més cites es Albert Einstein (10 cites), següit per J. K. Rowling amb 1 cita menys i Marilyn Monroe amb 7 cites.

In [65]:
df_final.tail(10)

,text,author,birthdate,bio
89,“The reason I talk to myself is because I’m th...,George Carlin,"May 12, 1937",George Denis Patrick Carlin was a Grammy-winni...
90,"“You may say I'm a dreamer, but I'm not the on...",John Lennon,"October 09, 1940","John Winston Ono Lennon, MBE, was an English s..."
91,“I am free of all prejudice. I hate everyone e...,W.C. Fields,"January 29, 1880",W. C. Fields was born William Claude Dukenfiel...
92,“The question isn't who is going to let me; it...,Ayn Rand,"February 02, 1905",Alisa Rosenbaum was born in pre-revolutionary ...
93,“I'm the one that's got to die when it's time ...,Jimi Hendrix,"November 27, 1942","Jimi Hendrix was an American guitarist, singer..."
94,“To die will be an awfully big adventure.”,J.M. Barrie,"May 09, 1860","Sir James Matthew Barrie, 1st Baronet, OM (9 M..."
95,“It takes courage to grow up and become who yo...,E.E. Cummings,"October 14, 1894","Edward Estlin Cummings was born in Cambridge, ..."
96,“But better to get hurt by the truth than comf...,Khaled Hosseini,"March 04, 1965","Hosseini was born in Kabul, Afghanistan, in 19..."
97,“You never really understand a person until yo...,Harper Lee,"April 28, 1926","Harper Lee, known as Nelle, was born in the Al..."
98,“You have to write the book that wants to be w...,Madeleine L'Engle,"November 29, 1918",Madeleine L'Engle was an American writer best ...


In [59]:
df_final.author.value_counts().head()

Albert Einstein    10
J.K. Rowling        9
Marilyn Monroe      7
Dr. Seuss           6
Mark Twain          6
Name: author, dtype: int64

In [60]:
df_final.author.nunique()

49